In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys
import urllib3
import json
import operator
import collections

import CPTAC.Endometrial as CPTAC

Welcome to the CPTAC data service package. Available datasets may be
viewed using CPTAC.list(). In order to access a specific data set,
import a CPTAC subfolder using either 'import CPTAC.Dataset' or 'from
CPTAC import Dataset'.
******
Version: 0.2.5
******
Loading Endometrial CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Acetylation Proteomics Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [2]:
somatic = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

# Part 1: Formatting Data

## Step 1: Enter gene name
# INPUT: 

In [3]:
gene = "ARID1A" #Name of any gene of which you are interested in examing the effects of mutation

## Step 2: Select type of data
Set "data_type" to "proteomics" for proteomic abundance data <br><br>
Set "data_type" to "phosphoproteomics" for phosphoproteomic data <br><br>
Set "data_type" to "transcriptomics" for transcription data
# INPUT:

In [4]:
data_type = "transcriptomics" #Options are "proteomics", "phosphoproteomics", "transcriptomics"

## Step 3: Select scope of data analysis
Set "scope" to "cis" to examine cis effect of gene mutation on the protein it codes for <br> <br>
Set "scope" to "interacting" to examine the effect of gene mutation on known interacting proteins (interacting proteins are automatically generated through interacting with STRING database API). If using scope="interacting", set "number_of_interacting_proteins" to the number of interacting proteins you want to pull from the STRING database <br><br>
Set "scope" to "all" to look at the effect on all proteins in our dataset
# INPUT:

In [5]:
scope = "interacting" #Options are "cis", "interacting", "all"
number_of_interacting_proteins = 20 #Parameter only necessary if examining "interacting", it will otherwise be ignored

## Step 4: Select Question

Set "question" to "mutated_vs_wildtype" to compare mutated and wildtype groups<br><br>
Set "question" to "hotspot_vs_wildtype" to compare individuals with a hotspot mutation <br><br>
Set "question" to "truncation_vs_wildtype" to compare individuals with a truncating mutation
# INPUT:

In [6]:
question = "mutated_vs_wildtype" #Options are .....

## Step 5: Set Dataframe to Selected Data

In [7]:
if data_type == "proteomics":
    dataframe = CPTAC.get_proteomics()
    
elif data_type == "phosphoproteomics":
    dataframe = CPTAC.get_phosphoproteomics()
    
elif data_type == "transcriptomics":
    dataframe = CPTAC.get_transcriptomics()
    
else:
    print("Invalid data_type value entered! See step 2 above.")

## Step 6: Generate Protein List Based on the Selected Scope

In [8]:
protein_list = []    

'''Just add the gene itself if cis'''
if scope == "cis":
    protein_list = [gene]
    
    '''If interacting, get interacting proteins from STRING database'''
elif scope == "interacting":
    
    '''Use urllib3 to access the string database api, gather list of interacting proteins'''
    urllib3.disable_warnings()
    string_api_url = "https://string-db.org/api"
    output_format = "json"
    method = "network"

    '''Use the specified gene and homo sapiens species code'''
    my_gene = [gene]
    species = "9606"

    '''Format the api request to collect the appropriate information'''
    request_url = string_api_url + "/" + output_format + "/" + method + "?"
    request_url += "identifiers=%s" % "%0d".join(my_gene)
    request_url += "&" + "species=" + species
    request_url += "&" + "limit=" + str(number_of_interacting_proteins)

    '''Send a request to the API, print the response status'''
    try:
        http = urllib3.PoolManager()
        response = http.request('GET',request_url)
        #print("Accessing String database, response status: ", response.status)
    except urllib3.HTTPError as err:
        error_message = err.read()
        print(error_message)
        sys.exit()

    interacting_proteins = []
    if response.status == 200: 
        '''Get the data from the API's response'''
        data = response.data
        y = json.loads(data)

        '''Make a list of the resulting interacting proteins'''
        for entry in y:
            if entry["preferredName_A"] not in interacting_proteins:
                interacting_proteins.append(entry["preferredName_A"])
            if entry["preferredName_B"] not in interacting_proteins:
                interacting_proteins.append(entry["preferredName_B"])
                
        for ip in interacting_proteins:
            protein_list.append(ip)
    else:
        print("\nSpecified gene was not found in String database, double check that you have it correctly!")
        protein_list.append(gene)

    '''If all, add all proteins in our dataset'''
elif scope == "all":
    protein_list = proteomics.columns

    '''If none of these, invalid'''
else:
    print("Invalid scope value entered! See step 3 above.")
    
'''Display list of proteins selected'''
print("Protein List: \n")
for pro in protein_list:
    print(pro)

Protein List: 

SMARCC1
CCND1
SMARCD3
CREBBP
SMARCB1
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
PHF10
ARID1B
NF1
CDC5L
SMARCD1
SMARCD2
SMARCA4
ACTL6A
SS18
DPF2
DPF3


## Step 7: Format Dataframe Based on Question

This will generate a dataframe with the gene mutation, sample status, and columns for either proteomics, phosphoproteomics, or transcriptomics for every protein (or every site in every protein, if phosphorylation).

#### Question: Mutated vs. Wildtype
Places either a 'Mutated' or 'Wildtype' value in the category column, based on gene mutation

In [9]:
if question == "mutated_vs_wildtype":
    '''Get dataframe for mutations, select sample status and gene mutation only'''
    genedf = CPTAC.compare_mutations(dataframe, protein_list[0], gene)
    genedf = genedf[[gene+"_Mutation", "Sample_Status"]]
    '''Loop through every protein in protein list, add in omics'''
    for protein in protein_list:
        protdf = CPTAC.compare_mutations(dataframe, protein, gene)
        if protdf is not None:
            for col in protdf.columns:
                if col not in genedf.columns:
                    genedf[col] = protdf[col]
                    
    '''Define category variable for mutated or wildtype'''                
    for ind, row in genedf.iterrows():
        if row[gene+"_Mutation"] != 'Wildtype_Tumor' and row[gene+"_Mutation"] != 'Wildtype':
            genedf.at[ind, 'Category'] = 'Mutated'
        else:
            genedf.at[ind, 'Category'] = 'Wildtype'
        

#### Question: Truncation Mutation vs Wildtype
Places either 'Truncating_Mutation', 'Non_Truncating_Mutation', or 'Wildtype' in the Category column, depending on gene mutation

In [10]:
if question == "truncation_vs_wildtype":
    '''Get dataframe for mutations, select sample status and gene mutation only'''
    genedf = CPTAC.compare_mutations(dataframe, gene, gene)
    genedf = genedf[[gene+"_Mutation", "Sample_Status"]]
    '''Loop through every protein in protein list, add in omics'''
    for protein in protein_list:
        protdf = CPTAC.compare_mutations(dataframe, protein, gene)
        if protdf is not None:
            for col in protdf.columns:
                if col not in genedf.columns:
                    genedf[col] = protdf[col]
                    
    truncating_mutations = ['Nonsense_Mutation', 'Frame_Shift_Del','Frame_Shift_Ins',]
    for ind, row in genedf.iterrows():
        if row[gene+"_Mutation"] in truncating_mutations:
            genedf.at[ind, 'Category'] = 'Truncating_Mutation'
        elif row[gene+"_Mutation"] == 'Wildtype' or row[gene+"_Mutation"] == 'Wildtype_Tumor':
            genedf.at[ind, 'Category'] = 'Wildtype'
        else:
            genedf.at[ind, 'Category'] = 'Non_Truncating_Mutation'

#### Show formatting

In [11]:
genedf.head()

,ARID1A_Mutation,Sample_Status,SMARCC1_omics,CCND1_omics,SMARCD3_omics,CREBBP_omics,SMARCB1_omics,SMARCA2_omics,SMARCC2_omics,KMT2D_omics,...,NF1_omics,CDC5L_omics,SMARCD1_omics,SMARCD2_omics,SMARCA4_omics,ACTL6A_omics,SS18_omics,DPF2_omics,DPF3_omics,Category
S001,Missense_Mutation,Tumor,13.24,12.18,8.86,11.27,10.78,10.80,11.47,11.71,...,13.23,11.92,11.18,11.85,12.41,11.14,12.26,10.05,3.53,Mutated
S002,Nonsense_Mutation,Tumor,13.54,11.53,6.70,11.27,11.43,11.71,11.86,11.12,...,12.88,12.43,11.33,12.27,13.23,11.54,12.66,10.27,1.89,Mutated
S003,Wildtype_Tumor,Tumor,12.73,12.04,8.03,10.51,10.76,12.53,11.04,10.65,...,12.71,11.78,10.06,11.89,11.99,10.93,12.74,9.76,4.16,Wildtype
S005,Wildtype_Tumor,Tumor,12.65,10.73,8.27,11.11,10.55,11.07,11.49,12.13,...,13.16,11.97,10.22,11.49,12.10,10.70,12.21,9.72,3.97,Wildtype
S006,Wildtype_Tumor,Tumor,12.50,9.27,8.37,12.14,11.14,12.99,11.85,11.11,...,14.39,11.77,9.86,12.18,13.43,11.30,13.00,9.33,1.80,Wildtype


# Part 2: Comparison

## Step 1: Select Desired Information from Dataframe
Set "drop_non_tumor" to "True" to look only at tumor patients. Set it to false to keep them in our comparison.
# INPUT: 

In [12]:
'''Select whether or not you want to drop non-tumor patients'''
drop_non_tumor = True #Options are True, False

In [13]:
'''Select only the Tumor patients if they have selected "drop_non_tumor"'''
if drop_non_tumor:
    genedf = genedf.loc[genedf['Sample_Status'] == 'Tumor']
    
'''Drop Columns that arent "Category" or omics data'''
genedf = genedf.drop('Sample_Status', axis = 1)
genedf = genedf.drop(gene+"_Mutation", axis = 1)

## Step 2: Select Variables for Comparison
Set "variable_a" and "variable_b" to two of the Options listed below for comparison

In [14]:
possible_choices = list(set(genedf['Category']))
print("Options for comparison (and number of): \n")
for choice in possible_choices:
    choicedf = genedf.loc[genedf['Category']==choice]
    print(choice, " (", len(choicedf), ")")

Options for comparison (and number of): 

Mutated  ( 43 )
Wildtype  ( 52 )


# INPUT: 

In [15]:
'''select 2 variables to do comparison'''
variable_a = 'Mutated' #See options from list above
variable_b = 'Wildtype' #See options from list above

## Step 3: Collect P-Values

In [16]:
sites = genedf.columns
p_values = {}
a_df = genedf.loc[genedf['Category'] == variable_a]
b_df = genedf.loc[genedf['Category'] == variable_b]
for site in sites:
    if site != 'Category':
        test_stat, p_val = scipy.stats.ttest_ind(a_df[site], b_df[site])
        if not np.isnan(p_val):
            p_values[site] = p_val
          
print("P-Values: \n")
print("{\n" + "\n".join("{}: {}".format(k, v) for k, v in p_values.items()) + "\n}")

P-Values: 

{
SMARCC1_omics: 0.03818000998497696
CCND1_omics: 0.26465516183197113
SMARCD3_omics: 0.0023588510726379195
CREBBP_omics: 0.7858560144352269
SMARCB1_omics: 0.08032724495382437
SMARCA2_omics: 0.48533815358894716
SMARCC2_omics: 0.026144713069129223
KMT2D_omics: 0.35349552293806574
ARID1A_omics: 0.3785693405806204
SMARCE1_omics: 0.05451641386469496
PHF10_omics: 0.058071979893612666
ARID1B_omics: 0.1380664204429325
NF1_omics: 0.23381115168299504
CDC5L_omics: 0.6746533053987078
SMARCD1_omics: 0.690941006229377
SMARCD2_omics: 0.102644565523063
SMARCA4_omics: 0.15375805697493475
ACTL6A_omics: 0.8562335281707476
SS18_omics: 0.17161240053891882
DPF2_omics: 0.10964309201738814
DPF3_omics: 0.0036397779246435825
}


## Step 4: Adjust for multiple testing with a bonferroni correction

In [17]:
alpha = .05
bonferroni_cutoff = alpha / len(p_values)
print("Bonferroni Adjusted P-Value Cutoff for Significance: ")
print(bonferroni_cutoff)

Bonferroni Adjusted P-Value Cutoff for Significance: 
0.002380952380952381


## Step 5: Sort into significant and insignificant groups

In [18]:
significant = {}
insignificant = {}
for key, val in p_values.items():
    if val <= bonferroni_cutoff:
        significant[key] = val
    else:
        insignificant[key] = val
        
print("Significant Values: \n")
print("{\n" + "\n".join("{}: {}".format(k, v) for k, v in significant.items()) + "\n}")

print("\nInsignificant Values: \n")
print("{\n" + "\n".join("{}: {}".format(k, v) for k, v in insignificant.items()) + "\n}")

Significant Values: 

{
SMARCD3_omics: 0.0023588510726379195
}

Insignificant Values: 

{
SMARCC1_omics: 0.03818000998497696
CCND1_omics: 0.26465516183197113
CREBBP_omics: 0.7858560144352269
SMARCB1_omics: 0.08032724495382437
SMARCA2_omics: 0.48533815358894716
SMARCC2_omics: 0.026144713069129223
KMT2D_omics: 0.35349552293806574
ARID1A_omics: 0.3785693405806204
SMARCE1_omics: 0.05451641386469496
PHF10_omics: 0.058071979893612666
ARID1B_omics: 0.1380664204429325
NF1_omics: 0.23381115168299504
CDC5L_omics: 0.6746533053987078
SMARCD1_omics: 0.690941006229377
SMARCD2_omics: 0.102644565523063
SMARCA4_omics: 0.15375805697493475
ACTL6A_omics: 0.8562335281707476
SS18_omics: 0.17161240053891882
DPF2_omics: 0.10964309201738814
DPF3_omics: 0.0036397779246435825
}
